# biopython
- [biopython](https://github.com/biopython/biopython)
- FASTQ
- FASTA
- SAM
- BAM
- VCF
- BED

## FASTQ

In [18]:
# FASTQ by Biopython

from Bio.SeqIO.QualityIO import FastqPhredIterator, FastqPhredWriter
from Bio.SeqRecord import SeqRecord


def _opener(filename):
    if filename.endswith('.gz'):
        import gzip
        return gzip.open
    elif filename.endswith('.bz2'):
        import bz2
        return bz2.open
    elif filename.endswith('.lzma'):
        import lzma
        return lzma.open
    else:
        return open
    

def head(fpath, h = 10) :
    openfun = _opener(fpath)
    if openfun is not None :
        with openfun(fpath) as fh:
            try :
                it = FastqPhredIterator(fh)
                for i, r in enumerate(it) :
                    if i > 10:
                        break
                    else:
                        print("i={}, r={}".format(i,r))
                              
            except Exception as e:
                    print("Error : {}".format(str(e)))
                    
                    
                              
def gen(fpath, num = 0):
    openfun = _opener(fpath)
    if openfun is not None :
        with openfun(fpath) as fh:
            it = FastqPhredIterator(fh)
            for i, r in enumerate(it) :
                if i < num or num < 1:
                    yield  r
                else :
                    break
                    

def clip(src, head = 6, tail = 6):
    for r in src :
        yield r[head :- tail]
        
            
def sample(src, frac = 0.1) :
    import random
    percent = int(frac * 100)
    for r in src:
        if random.randrange(1,101) < percent:
            yield r
"""
    - "qual" means simple quality files using PHRED scores (e.g. from Roche 454)
    - "fastq" means Sanger style FASTQ files using PHRED scores and an ASCII
      offset of 33 (e.g. from the NCBI Short Read Archive and Illumina 1.8+).
      These can potentially hold PHRED scores from 0 to 93.
    - "fastq-sanger" is an alias for "fastq".
    - "fastq-solexa" means old Solexa (and also very early Illumina) style FASTQ
      files, using Solexa scores with an ASCII offset 64. These can hold Solexa
      scores from -5 to 62.
    - "fastq-illumina" means newer Illumina 1.3 to 1.7 style FASTQ files, using
      PHRED scores but with an ASCII offset 64, allowing PHRED scores from 0
      to 62.
      
>>> from Bio import SeqIO
>>> for record in SeqIO.parse("Quality/example.fastq", "fastq"):
...     print("%s %s" % (record.id, record.seq))

"""
def output(fpath, src) :
    with _opener(fpath)(fpath, "w") as out :
        writer = FastqPhredWriter(out)
        writer.write_header()
    
        for r in src:
            #print(r)
            writer.write_record(r)
            
            
def split(read1, read2, src) :
    with _opener(read1)(read1, "w") as f1:
        writer1 = FastqPhredWriter(f1)
        writer1.write_header()
        
        with _opener(read2)(read2, "w") as f2 :
            writer2 = FastqPhredWriter(f2)
            writer2.write_header()
            
            for r in src :
                l = int(len(r.seq)/2)
                #print(l)
                writer1.write_record(r[0:l])
                writer2.write_record(r[l:])
                
                
def stat(it) :
    import collections
    
    index = collections.defaultdict(int)
    for r in it:
        index[r.seq[0:4]]+=1
        index[r.seq[-4:]]+=1
    print("size={}".format(len(index)))
    for k in index.keys() :
        print("key={},v={}".format(k,index[k]))
        
    
'''
SRR3503011	SAMN04478409	CTR15	CTR15-c1-noPhiX
SRR3503012	SAMN04478409	CTR15	CTR15-c1
SRR3503013	SAMN04478410	CTR31	CTR31-c1-NextSeq
SRR3503014	SAMN04478410	CTR31	CTR31-c1-noPhiX
SRR3503015	SAMN04478410	CTR31	CTR31-c1
'''

num = 10000
src = gen("SRR3503015.fastq", num)
split("SRR15-{}_1.fastq".format(num), "SRR15-{}_2.fastq".format(num), src)

"""
for i, s in enumerate(src) :
    if i < 1 :
        print(s)
        print(s[6:-6])
        print(s.seq)
        print(s[6:-6].seq)
"""
        
#stat(clip(src))
#output("srr3503015_sample.fastq", clip(src))

#head("SRR3503015.fastq")

'\nfor i, s in enumerate(src) :\n    if i < 1 :\n        print(s)\n        print(s[6:-6])\n        print(s.seq)\n        print(s[6:-6].seq)\n'

In [17]:
from Bio.SeqIO.QualityIO import FastqPhredIterator, FastqPhredWriter
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from collections import defaultdict

ids = defaultdict(int)
with open("capp-seq/selector-clean.fa", "w") as handle:
    for r in SeqIO.parse("capp-seq/selector-orig.fa", "fasta"):
        desc = r.description.split(" ")
        r.id = desc[1]
        r.name = desc[1]
        ids[r.id] += 1
        if ids[r.id] >1 :
            continue
        #print(r.id)
        SeqIO.write(r, handle, "fasta")
        
for key in ids.keys() :
    if ids[key] > 1 :
        print("{}={}".format(key,ids[key]))

range=chr13:48916201-48917522=4
range=chr17:7576020-7577221=4


# BED

In [37]:
import os
from Bio import SeqIO
hg19_root = os.environ["BIOREF"] + "/chromFa-hg19"
#names = [str(i) for i in range(1,23)] + ["X" , "Y" , "M"]
names = [str(i) for i in range(1,3)]
chrs = ["chr" + i for i in names]
paths = { i: hg19_root + '/' + i + ".fa" for i in chrs}
print(list(paths.values()))
index_db = SeqIO.index_db(hg19_root+"/index_db.sqlite", list(paths.values()), "fasta")
#index = { i: SeqIO.index(paths[i], "fasta") for i in chrs}
#print(chrs)
#for i in chrs :
#    print( paths[i])
for k in index_db.keys() :
    print("{} len = {}", k , len(index_db[k].seq))#

['/home/tom/zdata/bioref/chromFa-hg19/chr1.fa', '/home/tom/zdata/bioref/chromFa-hg19/chr2.fa']
{} len = {} chr1 249250621
{} len = {} chr2 243199373


In [82]:
! cat srr3503015_sample.fastq

@SRR3503015.1 1 length=194
ATTCATTCAAGAAATGTTAAACATGCAAATTTTTAGCACATGTATATGTAATATTCACTTTACTACATATTGCAATGTACTTGAAGCTTTTATATTAGAAGACTTAATAATGAATGCTAATACCTTTCCTATATGAAGGATTTTTCATGATAGACATTGAAATATGAAATTTTGATGGTACTATTATTCCCATT
+
:DDFHHHHHJIIJIHJFCIHHIJCFHHIIIHIEGGFHHIIGIGDHIJIFFGGI>FHGHHEHHIFFHIFHFFCCHHIIDEHGIIJHHGFCEFBDEFFFADDDHHHHHIFHHHGIGIHHFHIGHIJHHHGIJJIGDCIFHIIEHIDFHIEHGHIIIID@FHGGHIGGGIIIJJJJCGHIHECHHGHFFFFD@ECEE
@SRR3503015.2 2 length=194
TGCGCCCTGCAGCGGCAGCAGGCGACGGCAGCCGAGAAGCCCTGGGATGCCAAGGCCCCCGAGGGCCGGCTGCGGAAGTGTGAGAGCACCGACTCGGCCCGGTCTTCCGGGGCCGCACGTTGTCCAGCTGGGCATCGTCCACCACCGCCTGGTTGTGGGAGATGAGCTGGGCGATGCGCTCCTCCAGCCGCTTC
+
DDEDHHHHHIJJJJJIJJJIHIGGGJHGEDDCC>B?BD<CBDCBDDDDDDDDDBDDDBBB;775BDB9BB><@<B<@B:A:ACD@C<AAA<<><@8?BDFFHHHHHJJHIJJJJIJJJJHIIJJJJHJJHHHHFEFFCCDDDDBBDBDD;BD:AD7?BDBD;ACCDDCDDDBBB<>BDB>BDCCD<ACDDD599
@SRR3503015.3 3 length=194
TGGCAGTCATAGTATGGTTCTTCTACCTGAATCTCTTGATCTTCACCTTCTTCTGGGTCTTCAATTCCCACACCGTCAGGCTCGTCGGCATCTCCCTGGCCGAGCAGCCAAATGGGGATGCTGATGAGCCAGATG

In [6]:
!ls /home/tom/zdata/bioref/chromFa-hg19 -l

total 3125152
-rw-rw-r-- 1 tom tom 138245449 Mar 21  2009 chr10.fa
-rw-rw-r-- 1 tom tom 137706654 Mar 21  2009 chr11.fa
-rw-rw-r-- 1 tom tom     40929 Mar 21  2009 chr11_gl000202_random.fa
-rw-rw-r-- 1 tom tom 136528940 Mar 21  2009 chr12.fa
-rw-rw-r-- 1 tom tom 117473283 Mar 21  2009 chr13.fa
-rw-rw-r-- 1 tom tom 109496538 Mar 21  2009 chr14.fa
-rw-rw-r-- 1 tom tom 104582027 Mar 21  2009 chr15.fa
-rw-rw-r-- 1 tom tom  92161856 Mar 21  2009 chr16.fa
-rw-rw-r-- 1 tom tom   1714462 Mar 21  2009 chr17_ctg5_hap1.fa
-rw-rw-r-- 1 tom tom  82819122 Mar 21  2009 chr17.fa
-rw-rw-r-- 1 tom tom     38271 Mar 21  2009 chr17_gl000203_random.fa
-rw-rw-r-- 1 tom tom     82960 Mar 21  2009 chr17_gl000204_random.fa
-rw-rw-r-- 1 tom tom    178103 Mar 21  2009 chr17_gl000205_random.fa
-rw-rw-r-- 1 tom tom     41845 Mar 21  2009 chr17_gl000206_random.fa
-rw-rw-r-- 1 tom tom  79638800 Mar 21  2009 chr18.fa
-rw-rw-r-- 1 tom tom      4371 Mar 21  2009 chr18_gl000207_random.fa
-rw-rw-r-- 1 tom tom  60311570 M